In [ ]:
!pip install gradio
!pip install ultralytics
!pip install -U ray[tune]


  Using cached ultralytics-8.3.41-py3-none-any.whl.metadata (35 kB)
  Using cached ultralytics_thop-2.0.12-py3-none-any.whl.metadata (9.4 kB)
Using cached ultralytics-8.3.41-py3-none-any.whl (899 kB)
Using cached ultralytics_thop-2.0.12-py3-none-any.whl (26 kB)
  Using cached ray-2.40.0-cp310-cp310-manylinux2014_x86_64.whl.metadata (17 kB)
  Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl.metadata (5.8 kB)
Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl (101 kB)
Using cached ray-2.40.0-cp310-cp310-manylinux2014_x86_64.whl (66.8 MB)


NameError: name 'clear_output' is not defined

In [ ]:
import torch
from PIL import Image
import re
import numpy as np
from IPython.display import display, Math, Latex
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from tqdm import tqdm
import json
from torch.utils.data import Dataset, DataLoader


import pandas as pd
import os, sys

# from PIL import Image
# from torchvision import transforms
# from sklearn.model_selection import train_test_split
# import sklearn as skl
# import torch as t

from ultralytics import YOLO
import matplotlib.pyplot as plt

import gradio as gr
import re

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
!gdown 1r1v0No6BL9rcj_V8NH80V2jejREqud0E

Downloading...
From (original): https://drive.google.com/uc?id=1r1v0No6BL9rcj_V8NH80V2jejREqud0E
From (redirected): https://drive.google.com/uc?id=1r1v0No6BL9rcj_V8NH80V2jejREqud0E&confirm=t&uuid=b324f2eb-4613-41f3-85f7-47cc829543e4
To: /content/30_best.pt
100% 52.1M/52.1M [00:01<00:00, 27.9MB/s]


In [ ]:
!gdown 1GS456miaCEOfG3Jqms2cH7V6ZYnhsVVR

Downloading...
From (original): https://drive.google.com/uc?id=1GS456miaCEOfG3Jqms2cH7V6ZYnhsVVR
From (redirected): https://drive.google.com/uc?id=1GS456miaCEOfG3Jqms2cH7V6ZYnhsVVR&confirm=t&uuid=9dda9139-ca39-4dce-a0cf-2ede1743b345
To: /content/trocr_version_2.gz
100% 2.08G/2.08G [00:32<00:00, 64.3MB/s]


In [ ]:
!gunzip trocr_version_2.gz
!tar -xvf trocr_version_2

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# YOLO

In [ ]:
localize_model = YOLO("30_best.pt")

In [ ]:
def localize_image(image):
	with torch.no_grad():
		predictions = localize_model.predict(
			source=image,
			conf=0.1,
			iou=0.1,
			stream=True
		)

	for prediction in predictions:
		if len(prediction.boxes.xyxy):
			boxes = prediction.boxes.xyxy.cpu().numpy()
			scores = prediction.boxes.conf.cpu().numpy()

			# print(boxes)
			# print(scores)
			# print(bbox)
			idx = np.argsort(scores)
			x1, y1, x2, y2 = boxes[idx[-1]]

			print(boxes)
			result = image.crop([x1, y1, x2, y2])

			# plt.imshow(result)
			# plt.axis('off')  # Hide the axis
			# plt.show()

			return result

	return image

# TrOCR

In [ ]:
# path = "/kaggle/input/latex-ocr-implementation/models/trocr-large-finetuned-math-captions"

path = './models/latexocr-finetuned'
# path = 'microsoft/trocr-large-handwritten'
processor = TrOCRProcessor.from_pretrained(path)
model = VisionEncoderDecoderModel.from_pretrained(path).to(device)

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.46.2"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 1024,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decod

In [ ]:
def predict_ocr(image):
	with torch.no_grad():
		pixel_values = processor(images = image, return_tensors="pt").pixel_values.to(device)
		out = model.generate(pixel_values, max_new_tokens = 256)
		pred = processor.decode(out[0], skip_special_tokens=True).replace("\\ ", "\\")
		symbols = ['lim', 'sin', 'cos', 'tan']
		for sym in symbols:
			pred = pred.replace(sym, f"\\{sym}")
		print(pred)
	return f'${pred}$'

## Ensemble

In [ ]:
def predict_latex(image):
	print('localizing image')
	cropped = localize_image(image)
	print('done localizing')
	print('predicting')
	prediction = predict_ocr(cropped)
	print('done prediction')

	return prediction

In [ ]:
# Create Gradio interface
with gr.Blocks() as demo:
	gr.Markdown("# Image to LaTeX Converter")
	gr.Markdown("Upload an image containing a mathematical expression, and this tool will convert it to LaTeX.")

	with gr.Row():
		with gr.Column():
			image_input = gr.Image(type="pil", label="Upload Image")
		with gr.Column():
			latex_output = gr.Markdown(height=100, latex_delimiters=[ {"left": "$", "right": "$", "display": True }])

	convert_button = gr.Button("Convert")

	convert_button.click(
		predict_latex,
		inputs=image_input,
		outputs=latex_output
	)

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ab26f010a740ced9b6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


localizing image

0: 768x1024 1 formula, 69.9ms
[[     370.23      401.74        1862      968.12]]
done localizing
predicting
\lim_{n\rightarrow\infty}\left(\frac{1}{n}+\frac{1}{n+1}+\frac{1}{n+2}\right)
done prediction
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ab26f010a740ced9b6.gradio.live


In [ ]:
# This part is the implementation of augmentation
import albumentations as A
import cv2
import numpy as np
import random


def add_padding(image, top, bottom, left, right, color=(255, 255, 255)):
	padded_image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
	return padded_image

def create_notebook_background(image, line_color=(200, 200, 200), line_thickness=1, line_spacing=random.randint(100, 200)):
	height, width, _ = image.shape
	background = np.ones_like(image) * 255  # Create a white background

	# Draw vertical lines
	for y in range(0, height, line_spacing):
		cv2.line(background, (0, y), (width, y), line_color, thickness=line_thickness)

	return background

def replace_white_pixel_with_background(image, background):
	# Find all white pixels in the image
	white_pixels = np.all(image == [255, 255, 255], axis=-1)

	# Replace white pixels with the corresponding pixels from the background
	image[white_pixels] = background[white_pixels]

	return image

def apply_transformations(image):
	transform = A.Compose([
		A.PlanckianJitter(temperature_limit=(5500, 9000) ,p=1),  # 10% chance to apply Planckian jitter
		A.Rotate(limit=(-10, 10), p=0.5),  # 50% chance to rotate the image
		A.Downscale(scale_min=0.5, scale_max=0.9, p=0.2),  # 20% chance to downscale the image
		A.OneOf([
			A.Blur(blur_limit=(3, 7), p=1),  # apply blur
			A.GlassBlur(sigma=0.8, max_delta=2, iterations=1, mode='fast', p=1),  # apply glass blur
			A.MedianBlur(blur_limit=(3,5), p=1),  # apply median blur
			A.Downscale(scale_min=0.5, scale_max=0.9, p=1), # downscale the image
		], p=1),  # 50% chance to apply one of the above blur augmentations
			A.RandomBrightnessContrast(brightness_limit=(-0.7,0.1), contrast_limit=(-0.2,0.7), p=1),  # 40% chance to adjust brightness and contrast
		A.OneOf([
			A.PlanckianJitter(temperature_limit=(5500, 9000), p=1),  # 10% chance to apply Planckian jitter
			A.RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, p=1),  # 30% chance to shift the RGB channels
		], p=0.5),  # 50% chance to apply one of the above color augmentations
		A.Posterize(num_bits=4, p=0.3),  # 30% chance to apply posterization
		A.Sharpen(alpha=(0.1, 1), lightness=(1.0, 1.0), p=0.3),  # 30% chance to sharpen the image
	])

	# Apply the augmentations to the input image
	augmented = transform(image=image)
	image = augmented["image"]
	# print(augmented['replay'])

	return image

def erode_and_dilate(image):

	pick = random.randint(0,1)
	if pick == 0:
		size= random.randint(1,4)
		kernel = np.ones((size, size), np.uint8)
		image = cv2.erode(image, kernel, iterations=1)
		# image= cv2.dilate(image, kernel, iterations=1)
	else:
		size= random.randint(1,2)
		kernel = np.ones((size, size), np.uint8)
		# image = cv2.erode(image, kernel, iterations=1)
		image= cv2.dilate(image, kernel, iterations=1)

	return image

def augment_image(image):
	# Read an example image
	image = cv2.imread("./converted_image.jpg")
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
	# Apply augmentations
	image = add_padding(image, 70, 70, 70, 70)
	if random.random() < 0.4:
		background = create_notebook_background(image)
		image = replace_white_pixel_with_background(image, background)
	image = augment_image(image)
	image = erode_and_dilate(image)

	return image

In [ ]:
import random

def crop_to_formula(image, padding = 30):



	# Image: 4 channel image with alpha.
	# Convert black pixels to white pixels.
	data = np.array(image)
	red, green, blue, alpha = data.T
	black_areas = (red < 10) & (blue < 10) & (green < 10)
	# Convert alpha to white.
	data[..., -1] = 255
	# Crop a box around the area that contains black pixels.
	coords = np.argwhere(black_areas)
	x0, y0 = coords.min(axis=0)
	x1, y1 = coords.max(axis=0) + 1
	# Add padding.
	x0 = max(0, x0 - padding)
	y0 = max(0, y0 - padding)
	x1 = min(image.width, x1 + padding)
	y1 = min(image.height, y1 + padding)
	print(data[y0:y1,x0:x1].shape)
	image = Image.fromarray(data[y0:y1, x0:x1])

	if random.random() < 0.5:
		image = np.array(image)
		image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

		# Apply augmentations
		# image = add_padding(image, 70, 70, 70, 70)
		background = create_notebook_background(image)
		image = replace_white_pixel_with_background(image, background)
		image = apply_transformations(image)
		if random.random() < 0.5:
			image = 255 - image

		image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
		image = Image.fromarray(image)

	return image.convert('RGB')

def renderedLaTeXLabelstr2Formula(label: str):
	# We're matching \\label{...whatever} and removing it
	label = label.replace("(", r"\left(")
	label = label.replace("[", r"\left[")
	label = label.replace("\{", r"\left\{")
	label = label.replace(")", r"\right)")
	label = label.replace("]", r"\right]")
	label = label.replace("\}", r"\right\}")
	# label = re.sub(r"\\label\{[^\}]*\}", "", label)
	# # We match \, and remove it.
	# label = re.sub(r"\\,", "", label)
	# label = re.sub(r"\\mbox\s*\{\s*(.*?)\s*\}", r"\1", label)
	# if r"\begin" in label:
	#	 label = re.sub(r"&(\d)", r"& \1", label)
	#	 label = re.sub(r"\\(\d)", r"\\ \1", label)
	#	 label = re.sub(r"matrix}(.*?)", r"matrix} \1", label)
	#	 label = re.sub(r"(\d)\\", r"\1 \\", label)
	return label

def display_formula(latex: str):
	# Remove \mbox{...} - not supported by the inline MathJax renderer
	parsed_latex = re.sub(r"\\mbox\{[^\}]*\}", "", latex)
	display(Math(parsed_latex))

In [ ]:


class renderedLaTeXDataset(Dataset):
	def __init__(self, image_folder, image_label_file, processor, device):
		self.image_folder = image_folder
		with open(image_label_file, 'r') as f:
			self.image_label = json.load(f)
		self.file_name = os.listdir(self.image_folder)
		self.device = device
		self.processor = processor
		self.num_image = len(self.image_label)

	def __len__(self):
		return self.num_image

	def __getitem__(self, idx):
		# img_name = os.path.join(self.image_folder, self.file_name[idx])
		num_folder = str(idx // 1000 + 1).zfill(3)
		img_name = os.path.join(self.image_folder, f"folder_{num_folder}/{idx+1}.jpg")
		# print(img_name)
		image = Image.open(img_name).convert('RGBA')
		image = crop_to_formula(image)
		display(image)


		inputs = self.processor(images = image, padding = "max_length", return_tensors="pt")

		# outside = np.array(inputs['pixel_values'])
		# print(outside)
		# display(Image.fromarray(outside))
		# display(Image.fromarray(np.array(inputs['pixel_values'].cpu())))

		for key in inputs:
			inputs[key] = inputs[key].squeeze() # Get rid of batch dimension since the dataloader will batch it for us.

		# formula_idx = self.image_label[(self.file_name[idx]).split(".")[0]]
		formula_idx = self.image_label[str(idx+1)]


		# print(formula_idx)

		formula_idx = renderedLaTeXLabelstr2Formula(formula_idx)
		print(formula_idx)
		display(Math(formula_idx))
		# print(caption)
		# print(self.processor.tokenizer.tokenize(caption))
		caption = self.processor.tokenizer.encode(
			formula_idx, return_tensors="pt", padding = "max_length", max_length = 512, truncation = True, # Tweak this
			).to(self.device).squeeze()
		print(inputs['pixel_values'].shape)
		return inputs, caption

def set_seed(seed):
	np.random.seed(seed)
	t.manual_seed(seed)
	if t.cuda.is_available():
		t.cuda.manual_seed_all(seed)
	skl.utils.check_random_state(seed)

In [ ]:
os.listdir('/kaggle/input/dataset-100k-images-trocr/train_splits/folder_001')[0]

In [ ]:
NUM_EPOCHS = 5
LEARNING_RATE = 1e-5
BATCH_SIZE = 2 # 10 gigs of Vram -> 4, <5 gigs of vram -> 2
SHUFFLE_DATASET = True

set_seed(0)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

train_ds = renderedLaTeXDataset(image_folder='/kaggle/input/dataset-100k-images-trocr/train_splits',
							   image_label_file='/kaggle/input/dataset-100k-images-trocr/train_splits/labels.json',
								processor=processor,
							   device=device)

train_dl = DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle = SHUFFLE_DATASET, num_workers = 0)
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.to(device)
model.train()

In [ ]:
idx = 23

image_ds, caption_ds = train_ds.__getitem__(idx)
# img_name = os.path.join(train_ds.image_folder, train_ds.file_name[idx])
# image = Image.open(img_name).convert('RGBA')

# import json
# with open('/kaggle/input/trocr-large-finetuned/pytorch/default/1/models/trocr-large-finetuned-math-captions/vocab.json', 'r') as file:
#	 tokenizer_json = json.load(file)

In [ ]:
history = []; val_history = []; val_timesteps = []
ema_loss = None; ema_alpha = 0.95
scaler = torch.amp.GradScaler(enabled = True)
for epoch in range(NUM_EPOCHS):
	with tqdm(train_dl, desc=f"Epoch {epoch + 1}/{NUM_EPOCHS}") as pbar:
		for batch, captions in pbar:
			pixel_values = batch["pixel_values"]

			optimizer.zero_grad()
			with torch.autocast(device_type = "cuda", dtype = t.float16, enabled = True):
				outputs = model(pixel_values = pixel_values,
								labels = captions)
				loss = outputs.loss
				history.append(loss.item())
			scaler.scale(loss).backward()
			scaler.step(optimizer)
			scaler.update()

			if ema_loss is None: ema_loss = loss.item()
			else: ema_loss = ema_loss * ema_alpha + loss.item() * (1 - ema_alpha)
			pbar.set_postfix(loss=ema_loss)


In [ ]:
idx = 1559
image = Image.open(f"/kaggle/input/latex-ocr/training-data/{idx}.jpg").convert('RGB')

from IPython.display import display, Math, Latex
display(image)
pixel_values = processor(images = image, return_tensors="pt").pixel_values.to(device)
out = model.generate(pixel_values, max_new_tokens = 256)
print("Prediction:")
pred = processor.decode(out[0], skip_special_tokens=True).replace("\\ ", "\\")
# print(pred)
display(Math(pred))

In [ ]:
!pip install gradio

In [ ]:
latex_data = r'\frac{b_{1}^{2}\frac{72^{2}+96+2_{1}}{c_{1}B_{1}+{t_{5}+b_{1}}}}{7\sqrt{28x+t_{1}}'

symbols = ['lim','sin','cos','tan']
for sym in symbols:
	latex_data = latex_data.replace(sym, f"\\{sym}")


# latex_data = re.sub(r'\b(sin|cos|tan|lim)\b', r'\\\1', latex_data)
print(latex_data)
display(Math(latex_data))

In [ ]:
model.save_pretrained("models/trocr-large-finetuned-math-captions")
processor.save_pretrained("models/trocr-large-finetuned-math-captions")
# t.save(history, "../models/trocr-large-finetuned-math-captions/history.pt")
# t.save(val_history, "../models/trocr-large-finetuned-math-captions/val_history.pt")
# t.save(val_timesteps, "../models/trocr-large-finetuned-math-captions/val_timesteps.pt")